# Fake News Detection Data Loader + SQLite Insertion Script (Google Colab Compatible)

This notebook loads raw fake and real news articles from CSV files, applies basic validation, assigns unique article IDs, standardizes subjects, and inserts the cleaned dataset into a SQLite database (`fakenews.db`). This serves as the foundational pipeline for downstream analysis, modeling, and visualization.


This notebook is the **first** in a series of four that together form a complete fake news detection pipeline. It prepares and loads the validated data into a SQLite database for downstream use.

# Import Required Libraries

In [ ]:
import pandas as pd
import os
from sqlalchemy import create_engine
from sqlalchemy import text
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Define base path and data file locations
base_path = '/content/drive/MyDrive/IS619_Project'
real_file = os.path.join(base_path, 'True.csv')
fake_file = os.path.join(base_path, 'Fake.csv')

# Optional: SQLite database file in Google Drive for persistence
sqlite_db_path = os.path.join(base_path, 'fakenews.db')
engine = create_engine(f'sqlite:///{sqlite_db_path}')

# Validation Functions
def is_valid_date(val):
    try:
        pd.to_datetime(val)
        return True
    except Exception:
        return False

def is_valid_subject(val):
    return isinstance(val, str) and val.strip() != ''

def is_valid_flag(val):
    return val in ['FAKE', 'REAL', 'SATIRE']

# Clean + Insert Function
subject_map = {
    'Government News': 'GOVERNMENT',
    'Middle-east': 'MIDDLE_EAST',
    'News': 'NEWS',
    'US_News': 'US_NEWS',
    'left-news': 'LEFT_NEWS',
    'politics': 'POLITICS',
    'politicsNews': 'POLITICS',
    'worldnews': 'NEWS'
}

def clean_and_insert(df, flag_value):
    df['fakeNews'] = flag_value

    # Filter valid rows
    valid_mask = (
        df['date'].apply(is_valid_date) &
        df['subject'].apply(is_valid_subject) &
        df['fakeNews'].apply(is_valid_flag)
    )
    df_clean = df[valid_mask].copy()

    # Clean and transform
    df_clean.columns = df_clean.columns.str.strip()
    df_clean['subject'] = df_clean['subject'].apply(lambda x: subject_map.get(x.strip(), 'UNKNOWN'))
    df_clean['published_date'] = pd.to_datetime(df_clean['date'], errors='coerce')
    df_clean['is_fake'] = df_clean['fakeNews'].apply(lambda x: x.strip().upper() == 'FAKE')

    # Add unique article_id: prefix with 'f' for FAKE, 'r' for REAL
    prefix = 'f' if flag_value == 'FAKE' else 'r'
    df_clean['article_id'] = [f"{prefix}{i}" for i in range(len(df_clean))]

    # Reorder and rename columns to match database schema
    df_clean = df_clean[['article_id', 'title', 'text', 'subject', 'date', 'fakeNews']]
    df_clean.columns = ['article_id', 'title', 'body', 'subject', 'date', 'fakeNews']

    # Insert into SQLite
    print(f"Inserting {len(df_clean)} rows into SQLite database...")
    df_clean.to_sql('news_article', engine, if_exists='append', index=False, method='multi')
    print("Insert complete.\n")

    return df_clean

# Drop existing table to avoid duplicate article_id insertions
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS news_article;"))
    print("Dropped existing 'news_article' table.")

# Run for both datasets
df_real = pd.read_csv(real_file)
print(f"\nLoaded REAL dataset from: {real_file}")
print("First 20 rows of REAL news data:")
print(df_real.head(20))
clean_and_insert(df_real, 'REAL')

df_fake = pd.read_csv(fake_file)
print(f"\nLoaded FAKE dataset from: {fake_file}")
print("First 20 rows of FAKE news data:")
print(df_fake.head(20))
clean_and_insert(df_fake, 'FAKE')

# Create an index on article_id for faster lookups and joins
with engine.connect() as conn:
    conn.execute(text(
        "CREATE UNIQUE INDEX IF NOT EXISTS idx_article_id ON news_article(article_id);"
    ))
    print("Created index on 'article_id' column.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dropped existing 'news_article' table.

Loaded REAL dataset from: /content/drive/MyDrive/IS619_Project/True.csv
First 20 rows of REAL news data:
                                                title  \
0   As U.S. budget fight looms, Republicans flip t...   
1   U.S. military to accept transgender recruits o...   
2   Senior U.S. Republican senator: 'Let Mr. Muell...   
3   FBI Russia probe helped by Australian diplomat...   
4   Trump wants Postal Service to charge 'much mor...   
5   White House, Congress prepare for talks on spe...   
6   Trump says Russia probe will be fair, but time...   
7   Factbox: Trump on Twitter (Dec 29) - Approval ...   
8          Trump on Twitter (Dec 28) - Global Warming   
9   Alabama official to certify Senator-elect Jone...   
10  Jones certified U.S. Senate winner despite Moo...   
11  New York governor questions the consti